In [1]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from random import normalvariate
from mpl_toolkits.mplot3d import Axes3D
from scipy.signal import find_peaks
from scipy import fftpack,signal
from scipy.linalg import expm
from scipy.linalg import logm
from scipy.special import jv
from scipy import interpolate, integrate
from scipy.integrate import complex_ode,simpson
from scipy import special
from scipy.optimize import curve_fit
from sympy.physics.wigner import wigner_3j ,wigner_6j
#from maxwellbloch import mb_solve, field, ob_atom
import matplotlib as mpl
from scipy.integrate import quad

In [2]:
def getRaman1(o1, s1, m1, n1, l1):  # 找 M1 sin(kx) cos(ky) cos(kz) taux sigmax 对应的耦合
    if o1 == 'g':
        o2 = 'e'
        if s1 == 'up':
            s2 = 'down'
            coeff = 1
        elif s1 == 'down':
            s2 = 'up'
            coeff = 1
        else:
            raise ValueError("Invalid spin s1: must be 'up' or 'down'")
        key = []
        coefffinal = [] 
        for dx, dy, dz, coeffmoment in [
            (0, 0, 0, -1j * 1 / 8),     
            (0, 0, -1, -1j * 1 / 8),
            (0, -1, 0, -1j * 1 / 8),
            (0, -1, -1, -1j * 1 / 8),
            (-1, 0, 0, 1j * 1 / 8),     
            (-1, 0, -1, 1j * 1 / 8),
            (-1, -1, 0, 1j * 1 / 8),
            (-1, -1, -1, 1j * 1 / 8),
        ]:
            key.append((o2, s2, m1 + dx, n1 + dy, l1 + dz))
            coefffinal.append(coeff * coeffmoment)

    elif o1 == 'e':
        o2 = 'g'
        if s1 == 'up':
            s2 = 'down'
            coeff = 1
        elif s1 == 'down':
            s2 = 'up'
            coeff = 1
        else:
            raise ValueError("Invalid spin s1: must be 'up' or 'down'")
        key = []
        coefffinal = [] 
        for dx, dy, dz, coeffmoment in [
            (1, 1, 1, -1j * 1 / 8),     
            (1, 1, 0, -1j * 1 / 8),
            (1, 0, 1, -1j * 1 / 8),
            (1, 0, 0, -1j * 1 / 8),
            (0, 1, 1, 1j * 1 / 8),     
            (0, 1, 0, 1j * 1 / 8),
            (0, 0, 1, 1j * 1 / 8),
            (0, 0, 0, 1j * 1 / 8),
        ]:
            key.append((o2, s2, m1 + dx, n1 + dy, l1 + dz))
            coefffinal.append(coeff * coeffmoment)

    else:
        raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
    
    return key, coefffinal

    
def getRaman2(o1,s1,m1,n1,l1): #找 -M1 cos(kx) sin(ky) cos(kz) taux sigmay 对应的耦合
    if o1=='g':
        o2 = 'e'
        if s1 == 'up':
            s2 = 'down'
            coeff = -1j
        elif s1=='down':
            s2 = 'up'
            coeff = 1j
        else:
            raise ValueError("Invalid spin s1: must be 'up' or 'down'")
        key = []
        coefffinal=[] 
        for dx, dy, dz, coeffmoment in [
            (0, 0, 0, 1j * 1 / 8),     
            (0, 0, -1, 1j * 1 / 8),
            (0, -1, 0, -1j * 1 / 8),
            (0, -1, -1, -1j * 1 / 8),
            (-1, 0, 0, 1j * 1 / 8),     
            (-1, 0, -1, 1j * 1 / 8),
            (-1, -1, 0, -1j * 1 / 8),
            (-1, -1, -1, -1j * 1 / 8),
        ]:
            mx2 = m1 + dx
            ny2 = n1 + dy
            lz2 = l1 + dz
            key.append((o2, s2, mx2, ny2, lz2))
            coefffinal.append(coeff*coeffmoment)

    elif o1 == 'e':
        o2 = 'g'
        if s1 == 'up':
            s2 = 'down'
            coeff = -1j
        elif s1=='down':
            s2 = 'up'
            coeff = 1j
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        coefffinal=[] 
        for dx, dy, dz, coeffmoment in [
            (1, 1, 1, 1j * 1 / 8),     
            (1, 1, 0, 1j * 1 / 8),
            (1, 0, 1, -1j * 1 / 8),
            (1, 0, 0, -1j * 1 / 8),
            (0, 1, 1, 1j * 1 / 8),     
            (0, 1, 0, 1j * 1 / 8),
            (0, 0, 1, -1j * 1 / 8),
            (0, 0, 0, -1j * 1 / 8),
        ]:
            mx2 = m1 + dx
            ny2 = n1 + dy
            lz2 = l1 + dz
            key.append((o2, s2, mx2, ny2, lz2))
            coefffinal.append(coeff*coeffmoment)
    else:
        raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
    
    return key,coefffinal

def getRaman3(o1,s1,m1,n1,l1): #找 M2,0 cos(kx) cos(ky) sin(kz) taux sigmaz 对应的耦合
    if o1=='g':
        o2 = 'e'
        if s1 == 'up':
            s2 = 'up'
            coeff = 1
        elif s1=='down':
            s2 = 'down'
            coeff = -1
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        coefffinal=[] 
        for dx, dy, dz, coeffmoment in [
            (0, 0, 0, -1j * 1 / 8),     
            (0, 0, -1, 1j * 1 / 8),
            (0, -1, 0, -1j * 1 / 8),
            (0, -1, -1, 1j * 1 / 8),
            (-1, 0, 0, -1j * 1 / 8),     
            (-1, 0, -1, 1j * 1 / 8),
            (-1, -1, 0, -1j * 1 / 8),
            (-1, -1, -1, 1j * 1 / 8),
        ]:
            mx2 = m1 + dx
            ny2 = n1 + dy
            lz2 = l1 + dz
            key.append((o2, s2, mx2, ny2, lz2))
            coefffinal.append(coeff*coeffmoment)
    elif o1 == 'e':
        o2 = 'g'
        if s1 == 'up':
            s2 = 'up'
            coeff = 1
        elif s1=='down':
            s2 = 'down'
            coeff = -1 
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        coefffinal=[] 
        for dx, dy, dz, coeffmoment in [
            (1, 1, 1, -1j * 1 / 8),     
            (1, 1, 0, 1j * 1 / 8),
            (1, 0, 1, -1j * 1 / 8),
            (1, 0, 0, 1j * 1 / 8),
            (0, 1, 1, -1j * 1 / 8),     
            (0, 1, 0, 1j * 1 / 8),
            (0, 0, 1, -1j * 1 / 8),
            (0, 0, 0, 1j * 1 / 8),
        ]:
            mx2 = m1 + dx
            ny2 = n1 + dy
            lz2 = l1 + dz
            key.append((o2, s2, mx2, ny2, lz2))
            coefffinal.append(coeff*coeffmoment)
    else:
        raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
    return key,coefffinal

def getRaman4(o1,s1,m1,n1,l1): #找 M2,1 cos(kx) cos(ky) cos(kz) taux sigma0 对应的耦合
    if o1=='g':
        o2='e' #o2 = 'e'
        if s1 == 'up':
            s2 = 'up'
            coeff = 1
        elif s1=='down':
            s2 = 'down'
            coeff = 1
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        coefffinal=[] 
        for dx, dy, dz, coeffmoment in [
            (0, 0, 0,  1 / 8),     
            (0, 0, -1,  1 / 8),
            (0, -1, 0,  1 / 8),
            (0, -1, -1,  1 / 8),
            (-1, 0, 0,  1 / 8),     
            (-1, 0, -1, 1 / 8),
            (-1, -1, 0,  1/ 8),
            (-1, -1, -1, 1/ 8),
        ]:
            mx2 = m1 + dx
            ny2 = n1 + dy
            lz2 = l1 + dz
            key.append((o2, s2, mx2, ny2, lz2))
            coefffinal.append(coeff*coeffmoment)
    elif o1 == 'e':
        o2='g'#o2 = 'g'
        if s1 == 'up':
            s2 = 'up'
            coeff = 1
        elif s1=='down':
            s2 = 'down'
            coeff = 1
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        coefffinal=[] 
        for dx, dy, dz, coeffmoment in [
            (1, 1, 1,  1 / 8),     
            (1, 1, 0,  1 / 8),
            (1, 0, 1,  1 / 8),
            (1, 0, 0,  1 / 8),
            (0, 1, 1,  1 / 8),     
            (0, 1, 0, 1 / 8),
            (0, 0, 1,  1/ 8),
            (0, 0, 0, 1/ 8),
        ]:
            mx2 = m1 + dx
            ny2 = n1 + dy
            lz2 = l1 + dz
            key.append((o2, s2, mx2, ny2, lz2))
            coefffinal.append(coeff*coeffmoment)
    else:
        raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
    return key,coefffinal

def getBasis(numm,numn,numl):
    #q = np.array([qx,qy,qz])  
    truncm = list(range(-numm, numm + 1))
    truncn = list(range(-numn, numn + 1))
    truncl = list(range(-numl, numl + 1))
    basis = []
    index_map = {}
    idx = 0
    for o in ['g', 'e']:
        for s in ['up', 'down']:
            for mx in truncm:
                for ny in truncn:
                    for lz in truncl:
                        basis.append((o, s, mx, ny, lz))
                        index_map[(o, s, mx, ny, lz)] = idx
                        idx += 1
    return basis,index_map

def getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21,delta):
    basis,index_map = getBasis(numm,numn,numl)
    Nb = len(basis)
    H = np.zeros((Nb, Nb), dtype=complex)
    #for i, (o1, s1, (m1, n1, l1), (px1, py1, pz1)) in enumerate(basis):
    for i, (o1, s1, m1, n1, l1) in enumerate(basis):
        # 动能项
        
        if o1 == 'g':
            shift = np.array([0, 0, 0])
        elif o1 == 'e':
            shift = np.array([1, 1, 1])
        else:
            raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
        px1 = qx + 2 * m1 + shift[0]
        py1 = qy + 2 * n1 + shift[1]
        pz1 = qz + 2 * l1 + shift[2]
        kinetic = (1/2*px1**2 + 1/2*py1**2 + pz1**2) - 0.5*(V0+V0+Vz) #(0.5*px1**2 + 0.5*py1**2 + pz1**2) - 0.5*(V0+V0+Vz)
        H[i, i] += kinetic

        # Zeeman 项
        if o1 == 'g':
            H[i, i] += delta 
        elif o1 == 'e':
            H[i, i] -= delta 
        else:
            raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
        
        for dx in [-1, 1]:
            key = (o1, s1, m1 + dx, n1, l1)
            j = index_map.get(key)
            if j is not None:
                H[i, j] += -V0 / 4

        for dy in [-1, 1]:
            key = (o1, s1, m1, n1 + dy, l1)
            j = index_map.get(key)
            if j is not None:
                H[i, j] += -V0 / 4

        for dz in [-1, 1]:
            key = (o1, s1, m1, n1, l1 + dz)
            j = index_map.get(key)
            if j is not None:
                H[i, j] += -Vz / 4

        key1,coeff1 = getRaman1(o1,s1,m1,n1,l1)
        key2,coeff2 = getRaman2(o1,s1,m1,n1,l1)
        key3,coeff3 = getRaman3(o1,s1,m1,n1,l1)
        key4,coeff4 = getRaman4(o1,s1,m1,n1,l1)

        for keyi1,keyv1 in enumerate(key1):
            j = index_map.get(keyv1)
            if j is not None:
                H[i, j] += M1*coeff1[keyi1]  # 可根据 σ 的结构继续细化

        for keyi2,keyv2 in enumerate(key2):
            j = index_map.get(keyv2)
            if j is not None:
                H[i, j] += M1*coeff2[keyi2]

        for keyi3,keyv3 in enumerate(key3):
            j = index_map.get(keyv3)
            if j is not None:
                H[i, j] += M20*coeff3[keyi3]

        for keyi4,keyv4 in enumerate(key4):
            j = index_map.get(keyv4)
            if j is not None:
                H[i, j] += M21*coeff4[keyi4]
    return H

# 构造电流算符
def CurrentOperatorZ(qz,numm,numn,numl):
    basis,index_map = getBasis(numm,numn,numl)
    Nb = len(basis)
    H = np.zeros((Nb, Nb), dtype=complex)
    #for i, (o1, s1, (m1, n1, l1), (px1, py1, pz1)) in enumerate(basis):
    for i, (o1, s1, m1, n1, l1) in enumerate(basis):
        # 动能项
        
        if o1 == 'g':
            shift = np.array([0, 0, 0])
        elif o1 == 'e':
            shift = np.array([1, 1, 1])
        else:
            raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
        #px1 = qx + 2 * m1 + shift[0]
        #py1 = qy + 2 * n1 + shift[1]
        pz1 = qz + 2 * l1 + shift[2]
        kinetic = (2*pz1*1)  #(0.5*px1**2 + 0.5*py1**2 + pz1**2) - 0.5*(V0+V0+Vz)
        H[i, i] += kinetic
    return H

def CurrentOperatorX(qx,numm,numn,numl):
    basis,index_map = getBasis(numm,numn,numl)
    Nb = len(basis)
    H = np.zeros((Nb, Nb), dtype=complex)
    #for i, (o1, s1, (m1, n1, l1), (px1, py1, pz1)) in enumerate(basis):
    for i, (o1, s1, m1, n1, l1) in enumerate(basis):
        # 动能项
        
        if o1 == 'g':
            shift = np.array([0, 0, 0])
        elif o1 == 'e':
            shift = np.array([1, 1, 1])
        else:
            raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
        px1 = qx + 2 * m1 + shift[0]
        #py1 = qy + 2 * n1 + shift[1]
        #pz1 = qz + 2 * l1 + shift[2]
        kinetic = (px1*1)  #(0.5*px1**2 + 0.5*py1**2 + pz1**2) - 0.5*(V0+V0+Vz)
        H[i, i] += kinetic
    return H


def CurrentOperatorY(qy,numm,numn,numl):
    basis,index_map = getBasis(numm,numn,numl)
    Nb = len(basis)
    H = np.zeros((Nb, Nb), dtype=complex)
    #for i, (o1, s1, (m1, n1, l1), (px1, py1, pz1)) in enumerate(basis):
    for i, (o1, s1, m1, n1, l1) in enumerate(basis):
        # 动能项
        
        if o1 == 'g':
            shift = np.array([0, 0, 0])
        elif o1 == 'e':
            shift = np.array([1, 1, 1])
        else:
            raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
        #px1 = qx + 2 * m1 + shift[0]
        py1 = qy + 2 * n1 + shift[1]
        #pz1 = qz + 2 * l1 + shift[2]
        kinetic = (py1*1)  #(0.5*px1**2 + 0.5*py1**2 + pz1**2) - 0.5*(V0+V0+Vz)
        H[i, i] += kinetic
    return H


def spinoperatorZ(o1,s1,m1,n1,l1):  #tau_0 sigma_z
    if o1=='g':
        o2 = 'g'
        if s1 == 'up':
            s2 = 'up'
            coeff = 1
        elif s1=='down':
            s2 = 'down'
            coeff = -1
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        key.append((o2, s2, m1, n1, l1))
        coefffinal=[] 
        coefffinal.append(coeff)
    elif o1 == 'e':
        o2 = 'e'
        if s1 == 'up':
            s2 = 'up'
            coeff = 1
        elif s1=='down':
            s2 = 'down'
            coeff = -1
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        key.append((o2, s2, m1, n1, l1))
        coefffinal=[] 
        coefffinal.append(coeff)
    else:
        raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
    return key,coefffinal

def spinoperatorX(o1,s1,m1,n1,l1):  #tau_0 sigma_x
    if o1=='g':
        o2 = 'g'
        if s1 == 'up':
            s2 = 'down'
            coeff = 1
        elif s1=='down':
            s2 = 'up'
            coeff = 1
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        key.append((o2, s2, m1, n1, l1))
        coefffinal=[] 
        coefffinal.append(coeff)
    elif o1 == 'e':
        o2 = 'e'
        if s1 == 'up':
            s2 = 'down'
            coeff = 1
        elif s1=='down':
            s2 = 'up'
            coeff = 1
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        key.append((o2, s2, m1, n1, l1))
        coefffinal=[] 
        coefffinal.append(coeff)
    else:
        raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
    return key,coefffinal

def spinoperatorY(o1,s1,m1,n1,l1):  #tau_0 sigma_y
    if o1=='g':
        o2 = 'g'
        if s1 == 'up':
            s2 = 'down'
            coeff = -1j
        elif s1=='down':
            s2 = 'up'
            coeff = 1j
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        key.append((o2, s2, m1, n1, l1))
        coefffinal=[] 
        coefffinal.append(coeff)
    elif o1 == 'e':
        o2 = 'e'
        if s1 == 'up':
            s2 = 'dwon'
            coeff = -1j
        elif s1=='down':
            s2 = 'up'
            coeff = 1j
        else:
            raise ValueError("Invalid orbital s1: must be 'up' or 'down'")
        key = []
        key.append((o2, s2, m1, n1, l1))
        coefffinal=[] 
        coefffinal.append(coeff)
    else:
        raise ValueError("Invalid orbital o1: must be 'g' or 'e'")
    return key,coefffinal

def spinProjectionOperatorZ(numm,numn,numl):
    basis,index_map = getBasis(numm,numn,numl)
    Nb = len(basis)
    Operator = np.zeros((Nb, Nb), dtype=complex)
    for i, (o1, s1, m1, n1, l1) in enumerate(basis):
        key, coeff = spinoperatorZ(o1, s1, m1, n1, l1)
        for keyi3,keyv3 in enumerate(key):
            j = index_map.get(keyv3)
            if j is not None:
                Operator[i, j] += coeff[keyi3]
    return Operator

def spinProjectionOperatorX(numm,numn,numl):
    basis,index_map = getBasis(numm,numn,numl)
    Nb = len(basis)
    Operator = np.zeros((Nb, Nb), dtype=complex)
    for i, (o1, s1, m1, n1, l1) in enumerate(basis):
        key, coeff = spinoperatorX(o1, s1, m1, n1, l1)
        for keyi3,keyv3 in enumerate(key):
            j = index_map.get(keyv3)
            if j is not None:
                Operator[i, j] += coeff[keyi3]
    return Operator


def spinProjectionOperatorY(numm,numn,numl):
    basis,index_map = getBasis(numm,numn,numl)
    Nb = len(basis)
    Operator = np.zeros((Nb, Nb), dtype=complex)
    for i, (o1, s1, m1, n1, l1) in enumerate(basis):
        key, coeff = spinoperatorY(o1, s1, m1, n1, l1)
        for keyi3,keyv3 in enumerate(key):
            j = index_map.get(keyv3)
            if j is not None:
                Operator[i, j] += coeff[keyi3]
    return Operator




def getSpinsectorZ(qx,qy,qz,numm, numn, numl, V0, Vz, M1, M20, M21, delta):
    H = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21,delta)
    spinsector = spinProjectionOperatorZ(numm,numn,numl)

    Eband,eigvec = np.linalg.eigh(H)
    U = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            U[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate())@spinsector@(eigvec[:,j].reshape(len(H),1)))[0][0]
    Espin,Vu = np.linalg.eigh(U)
    Vup = Vu[0,1]*eigvec[:,0] + Vu[1,1]*eigvec[:,1]
    Vdown = Vu[0,0]*eigvec[:,0] + Vu[1,0]*eigvec[:,1]
    return Vup.reshape(len(H),1), Vdown.reshape(len(H),1), Eband[:10], Espin

def getSpinsectorX(qx,qy,qz,numm, numn, numl, V0, Vz, M1, M20, M21, delta):
    H = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21,delta)
    spinsector = spinProjectionOperatorX(numm,numn,numl)

    Eband,eigvec = np.linalg.eigh(H)
    U = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            U[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate())@spinsector@(eigvec[:,j].reshape(len(H),1)))[0][0]
    Espin,Vu = np.linalg.eigh(U)
    Vup = Vu[0,1]*eigvec[:,0] + Vu[1,1]*eigvec[:,1]
    Vdown = Vu[0,0]*eigvec[:,0] + Vu[1,0]*eigvec[:,1]
    return Vup.reshape(len(H),1), Vdown.reshape(len(H),1), Eband[:10], Espin


def getSpinsectorY(qx,qy,qz,numm, numn, numl, V0, Vz, M1, M20, M21, delta):
    H = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21,delta)
    spinsector = spinProjectionOperatorY(numm,numn,numl)

    Eband,eigvec = np.linalg.eigh(H)
    U = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            U[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate())@spinsector@(eigvec[:,j].reshape(len(H),1)))[0][0]
    Espin,Vu = np.linalg.eigh(U)
    Vup = Vu[0,1]*eigvec[:,0] + Vu[1,1]*eigvec[:,1]
    Vdown = Vu[0,0]*eigvec[:,0] + Vu[1,0]*eigvec[:,1]
    return Vup.reshape(len(H),1), Vdown.reshape(len(H),1), Eband[:10], Espin





In [3]:
def nonAbeianUxy(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt):
    M21t = M21 * np.sin(t)
    delta = delta0 - deltam*np.cos(t)
    H =  getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkx = getHam(qx+dkx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta) 
    Hdky = getHam(qx,qy+dky,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta) 
    Hdkxdky = getHam(qx+dkx,qy+dky,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkx = np.linalg.eigh(Hdkx)
    _,eigvecdky = np.linalg.eigh(Hdky)
    _,eigvecdkxdky = np.linalg.eigh(Hdkxdky)

    Ux = np.zeros((2,2),dtype=complex)
    Uyx = np.zeros((2,2),dtype=complex)
    Uy = np.zeros((2,2),dtype=complex)
    Uxy = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Ux[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkx[:,j].reshape(len(H),1)))[0][0]
            Uyx[i,j] = (((eigvecdkx[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkxdky[:,j].reshape(len(H),1)))[0][0]
            Uxy[i,j] = (((eigvecdky[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkxdky[:,j].reshape(len(H),1)))[0][0]
            Uy[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdky[:,j].reshape(len(H),1)))[0][0]
    U = Ux@Uyx@np.linalg.inv(Uxy)@np.linalg.inv(Uy)
    return logm(U)

def nonAbeianUzw(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt):
    delta = delta0 - deltam*np.cos(t)
    M21t = M21 * np.sin(t)
    deltadt = delta0 - deltam*np.cos(t+dt)
    M21tdt = M21 * np.sin(t+dt)
    H = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkz = getHam(qx,qy,qz+dkz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkw = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21tdt,deltadt)
    Hdkzdkw = getHam(qx,qy,qz+dkz,numm,numn,numl,V0,Vz,M1,M20,M21tdt,deltadt)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkz = np.linalg.eigh(Hdkz)
    _,eigvecdkw = np.linalg.eigh(Hdkw)
    _,eigvecdkzdkw = np.linalg.eigh(Hdkzdkw)
    Uz = np.zeros((2,2),dtype=complex)
    Uwz = np.zeros((2,2),dtype=complex)
    Uw = np.zeros((2,2),dtype=complex)
    Uzw = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uz[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkz[:,j].reshape(len(H),1)))[0][0]
            Uwz[i,j] = (((eigvecdkz[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkzdkw[:,j].reshape(len(H),1)))[0][0]
            Uzw[i,j] = (((eigvecdkw[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkzdkw[:,j].reshape(len(H),1)))[0][0]
            Uw[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkw[:,j].reshape(len(H),1)))[0][0]
    U = Uz@Uwz@np.linalg.inv(Uzw)@np.linalg.inv(Uw)
    return logm(U)

def nonAbeianUwx(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt):
    delta = delta0 - deltam*np.cos(t)
    M21t = M21 * np.sin(t)
    deltadt = delta0 - deltam*np.cos(t+dt)
    M21tdt = M21 * np.sin(t+dt)
    H = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkx = getHam(qx+dkx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkw = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21tdt,deltadt)
    Hdkxdkw = getHam(qx+dkx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21tdt,deltadt)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkx = np.linalg.eigh(Hdkx)
    _,eigvecdkw = np.linalg.eigh(Hdkw)
    _,eigvecdkxdkw = np.linalg.eigh(Hdkxdkw)
    Uw = np.zeros((2,2),dtype=complex)
    Uxw = np.zeros((2,2),dtype=complex)
    Uwx = np.zeros((2,2),dtype=complex)
    Ux = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uw[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkw[:,j].reshape(len(H),1)))[0][0]
            Uwx[i,j] = (((eigvecdkx[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkxdkw[:,j].reshape(len(H),1)))[0][0]
            Uxw[i,j] = (((eigvecdkw[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkxdkw[:,j].reshape(len(H),1)))[0][0]
            Ux[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkx[:,j].reshape(len(H),1)))[0][0]
    U = Uw@Uxw@np.linalg.inv(Uwx)@np.linalg.inv(Ux)
    return logm(U)

def nonAbeianUzy(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt):
    delta = delta0 - deltam*np.cos(t)
    M21t = M21 * np.sin(t)
    H = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkz = getHam(qx,qy,qz+dkz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdky = getHam(qx,qy+dky,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkzdky = getHam(qx,qy+dky,qz+dkz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkz = np.linalg.eigh(Hdkz)
    _,eigvecdky = np.linalg.eigh(Hdky)
    _,eigvecdkzdky = np.linalg.eigh(Hdkzdky)
    Uz = np.zeros((2,2),dtype=complex)
    Uyz = np.zeros((2,2),dtype=complex)
    Uzy = np.zeros((2,2),dtype=complex)
    Uy = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uz[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkz[:,j].reshape(len(H),1)))[0][0]
            Uyz[i,j] = (((eigvecdkz[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkzdky[:,j].reshape(len(H),1)))[0][0]
            Uzy[i,j] = (((eigvecdky[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkzdky[:,j].reshape(len(H),1)))[0][0]
            Uy[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdky[:,j].reshape(len(H),1)))[0][0]
    U = Uz@Uyz@np.linalg.inv(Uzy)@np.linalg.inv(Uy)
    return logm(U)

def nonAbeianUzx(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt):
    delta = delta0 - deltam*np.cos(t)
    M21t = M21 * np.sin(t)
    H = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkz = getHam(qx,qy,qz+dkz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkx = getHam(qx+dkx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkxdkz = getHam(qx+dkx,qy,qz+dkz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdkz = np.linalg.eigh(Hdkz)
    _,eigvecdkx = np.linalg.eigh(Hdkx)
    _,eigvecdkxdkz = np.linalg.eigh(Hdkxdkz)
    Uz = np.zeros((2,2),dtype=complex)
    Uxz = np.zeros((2,2),dtype=complex)
    Uzx = np.zeros((2,2),dtype=complex)
    Ux = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uz[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkz[:,j].reshape(len(H),1)))[0][0]
            Uxz[i,j] = (((eigvecdkz[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkxdkz[:,j].reshape(len(H),1)))[0][0]
            Uzx[i,j] = (((eigvecdkx[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkxdkz[:,j].reshape(len(H),1)))[0][0]
            Ux[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkx[:,j].reshape(len(H),1)))[0][0]
    U = Uz@Uxz@np.linalg.inv(Uzx)@np.linalg.inv(Ux)
    return logm(U)

def nonAbeianUyw(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt):
    delta = delta0 - deltam*np.cos(t)
    M21t = M21 * np.sin(t)
    deltadt = delta0 - deltam*np.cos(t+dt)
    M21tdt = M21 * np.sin(t+dt)
    H = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdky = getHam(qx,qy+dky,qz,numm,numn,numl,V0,Vz,M1,M20,M21t,delta)
    Hdkw = getHam(qx,qy,qz,numm,numn,numl,V0,Vz,M1,M20,M21tdt,deltadt)
    Hdkydkw = getHam(qx,qy+dky,qz,numm,numn,numl,V0,Vz,M1,M20,M21tdt,deltadt)
    _,eigvec = np.linalg.eigh(H)
    _,eigvecdky = np.linalg.eigh(Hdky)
    _,eigvecdkw = np.linalg.eigh(Hdkw)
    _,eigvecdkydkw = np.linalg.eigh(Hdkydkw)
    Uy = np.zeros((2,2),dtype=complex)
    Uwy = np.zeros((2,2),dtype=complex)
    Uyw = np.zeros((2,2),dtype=complex)
    Uw = np.zeros((2,2),dtype=complex)
    for i in range(2):
        for j in range(2):
            Uy[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdky[:,j].reshape(len(H),1)))[0][0]
            Uwy[i,j] = (((eigvecdky[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkydkw[:,j].reshape(len(H),1)))[0][0]
            Uyw[i,j] = (((eigvecdkw[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkydkw[:,j].reshape(len(H),1)))[0][0]
            Uw[i,j] = (((eigvec[:,i].reshape(len(H),1)).T.conjugate()) @ (eigvecdkw[:,j].reshape(len(H),1)))[0][0]
    U = Uy@Uwy@np.linalg.inv(Uyw)@np.linalg.inv(Uw)
    return logm(U)

def FlP(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt):
    Uxy = nonAbeianUxy(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt)
    Uzw = nonAbeianUzw(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt)
    Uwx = nonAbeianUwx(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt)
    Uzy = nonAbeianUzy(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt)
    Uzx = nonAbeianUzx(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt)
    Uyw = nonAbeianUyw(qx, qy, qz, t, numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dt)
    return Uxy@Uzw + Uwx@Uzy + Uzx@Uyw
    




In [ ]:

V0,Vz,M1,M20,M21 = 2, 4, 1, 1, 0.15
numm, numn, numl = 2, 2, 2
deltam = 0.065*2
delta0 = 0.13
N= 15
kx = np.arange(-N,N,2)/N
ky = np.arange(-N,N,2)/N
kz = np.arange(-N,N,2)/N
kw = np.arange(-N,N,2)/N*np.pi

Fk = np.zeros((len(kx),len(ky),len(kz),len(kw),2,2),dtype=complex)
dkx = kx[1]-kx[0]
dky = ky[1]-ky[0]
dkz = kz[1]-kz[0]
dkw = kw[1]-kw[0]
for i in range(len(kx)):
    for j in range(len(ky)):
        for k in range(len(kz)):
            for l in range(len(kw)):
                Fk[i,j,k,l] = FlP(kx[i], ky[j], kz[k], kw[l], numm, numn, numl, V0, Vz, M1, M21, M20, delta0,deltam, dkx, dky, dkz, dkw)
Ftrace = np.zeros((len(kx),len(ky),len(kz),len(kw)),dtype=complex)
for i in range(len(kx)):
    for j in range(len(ky)):
        for k in range(len(kz)):
            for l in range(len(kw)):
                Ftrace[i,j,k,l] = np.trace(Fk[i,j,k,l])
1/(4*(np.pi)**2)*np.sum(Ftrace)

np.complex128(-2.7230158612596775-4.911756350259826e-16j)